In [ ]:
import pandas as pd
import tarfile
import io

df_mc_meta=pd.read_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.csv')

In [ ]:
df_mc_meta

In [ ]:
df_mc_meta.head(10).to_csv('/data2st1/junyi/methlyatlas/mCseq/CEMBA.mC.Metadata/CEMBA.mC.Metadata.head10.csv',index=False)

In [ ]:
df_meta_integrateion = pd.read_csv("/data2st1/junyi/methlyatlas/mCseq/mC-ATAC.Integration/mC-ATAC.Integration.csv")

In [ ]:
df_meta_integrateion.head(10)

In [44]:
import h5py
filename = "/data2st1/junyi/methlyatlas/atac/catlas.org/catlas_downloads/mousebrain/snap/CEMBA180123_2A.snap"

with h5py.File(filename, 'r') as file:
    # List all groups in the file
    print("Groups in the HDF5 file:")
    for group in file.keys():
        print(group)


BlockingIOError: [Errno 11] Unable to open file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')

In [ ]:
import snapatac2 as snap

snap.__version__


In [ ]:
adata=snap.read("/data2st1/junyi/methlyatlas/atac/catlas.org/catlas_downloads/mousebrain/snap/CEMBA180123_2A.snap",backend="hdf5")

In [ ]:
import snapatac2 as snap


In [ ]:
fragment_file = snap.datasets.pbmc5k()


In [ ]:
data = snap.pp.import_data(
    fragment_file,
    chrom_sizes=snap.genome.hg38,
    file="pbmc.h5ad",  # Optional
    sorted_by_barcode=False,
)
data
